## 📦 Importing required libraries


In [28]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report

## 📥 Reading the dataset

In [29]:
df = pd.read_csv('/kaggle/input/25-05-league-of-legends-champion-data-2025/140325_LoL_champion_data.csv')
df


,Unnamed: 0,id,apiname,title,difficulty,herotype,alttype,resource,stats,rangetype,...,be,rp,skill_i,skill_q,skill_w,skill_e,skill_r,skills,fullname,nickname
0,Aatrox,266.0,Aatrox,the Darkin Blade,2,Fighter,Tank,Blood Well,"{'hp_base': 650, 'hp_lvl': 114, 'mp_base': 0, ...",Melee,...,2400,880,{1: 'Deathbringer Stance'},"{1: 'The Darkin Blade', 2: 'The Darkin Blade 3'}",{1: 'Infernal Chains'},{1: 'Umbral Dash'},{1: 'World Ender'},"{1: 'Deathbringer Stance', 2: 'The Darkin Blad...",NaN,NaN
1,Ahri,103.0,Ahri,the Nine-Tailed Fox,2,Mage,Assassin,Mana,"{'hp_base': 590, 'hp_lvl': 104, 'mp_base': 418...",Ranged,...,1575,790,{1: 'Essence Theft'},{1: 'Orb of Deception'},{1: 'Fox-Fire'},{1: 'Charm'},{1: 'Spirit Rush'},"{1: 'Essence Theft', 2: 'Orb of Deception', 3:...",NaN,NaN
2,Akali,84.0,Akali,the Rogue Assassin,2,Assassin,NaN,Energy,"{'hp_base': 600, 'hp_lvl': 119, 'mp_base': 200...",Melee,...,1575,790,"{1: ""Assassin's Mark""}",{1: 'Five Point Strike'},{1: 'Twilight Shroud'},{1: 'Shuriken Flip'},{1: 'Perfect Execution'},"{1: ""Assassin's Mark"", 2: 'Five Point Strike',...",Akali Jhomen Tethi,NaN
3,Akshan,166.0,Akshan,the Rogue Sentinel,3,Marksman,Assassin,Mana,"{'hp_base': 630, 'hp_lvl': 107, 'mp_base': 350...",Ranged,...,2400,880,{1: 'Dirty Fighting'},{1: 'Avengerang'},{1: 'Going Rogue'},{1: 'Heroic Swing'},{1: 'Comeuppance'},"{1: 'Dirty Fighting', 2: 'Avengerang', 3: 'Goi...",NaN,NaN
4,Alistar,12.0,Alistar,the Minotaur,1,Tank,Support,Mana,"{'hp_base': 685, 'hp_lvl': 120, 'mp_base': 350...",Melee,...,675,585,{1: 'Triumphant Roar'},{1: 'Pulverize'},{1: 'Headbutt'},{1: 'Trample'},{1: 'Unbreakable Will'},"{1: 'Triumphant Roar', 2: 'Pulverize', 3: 'Hea...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Zeri,221.0,Zeri,The Spark of Zaun,2,Marksman,NaN,Mana,"{'hp_base': 600, 'hp_lvl': 110, 'mp_base': 250...",Ranged,...,2400,880,{1: 'Living Battery'},{1: 'Burst Fire'},{1: 'Ultrashock Laser'},{1: 'Spark Surge'},{1: 'Lightning Crash'},"{1: 'Living Battery', 2: 'Burst Fire', 3: 'Ult...",NaN,NaN
168,Ziggs,115.0,Ziggs,the Hexplosives Expert,2,Mage,NaN,Mana,"{'hp_base': 606, 'hp_lvl': 106, 'mp_base': 480...",Ranged,...,2400,880,{1: 'Short Fuse'},{1: 'Bouncing Bomb'},{1: 'Satchel Charge'},{1: 'Hexplosive Minefield'},{1: 'Mega Inferno Bomb'},"{1: 'Short Fuse', 2: 'Bouncing Bomb', 3: 'Satc...",NaN,NaN
169,Zilean,26.0,Zilean,the Chronokeeper,2,Support,Mage,Mana,"{'hp_base': 574, 'hp_lvl': 96, 'mp_base': 452,...",Ranged,...,675,585,{1: 'Time in a Bottle'},{1: 'Time Bomb'},{1: 'Rewind'},{1: 'Time Warp'},{1: 'Chronoshift'},"{1: 'Time in a Bottle', 2: 'Time Bomb', 3: 'Re...",NaN,NaN
170,Zoe,142.0,Zoe,the Aspect of Twilight,3,Mage,Support,Mana,"{'hp_base': 630, 'hp_lvl': 106, 'mp_base': 425...",Ranged,...,2400,880,{1: 'More Sparkles!'},"{1: 'Paddle Star', 2: 'Paddle Star 2'}",{1: 'Spell Thief'},{1: 'Sleepy Trouble Bubble'},{1: 'Portal Jump'},"{1: 'More Sparkles!', 2: 'Paddle Star', 3: 'Sp...",NaN,NaN


## 🧹 Data preparation and cleaning


In [30]:
columns_to_drop = ['apiname', 'title', 'alttype',  'stats', 'patch', 'changes', 'role',
    'client_positions', 'external_positions', 'adaptivetype', 'be', 'rp', 'skill_i', 'skill_q',
    'skill_e', 'skill_w', 'skill_r', 'skills', 'fullname', 'nickname', 'style',
]

df_cleaned = df.drop(columns=columns_to_drop)
df_cleaned.head()

,Unnamed: 0,id,difficulty,herotype,resource,rangetype,date,damage,toughness,control,mobility,utility
0,Aatrox,266.0,2,Fighter,Blood Well,Melee,2013-06-13,3,3,2,2,2
1,Ahri,103.0,2,Mage,Mana,Ranged,2011-12-14,3,1,2,3,1
2,Akali,84.0,2,Assassin,Energy,Melee,2010-05-11,3,1,1,3,1
3,Akshan,166.0,3,Marksman,Mana,Ranged,2021-07-22,3,1,1,3,2
4,Alistar,12.0,1,Tank,Mana,Melee,2009-02-21,1,3,3,1,2


In [31]:
duplicates = df_cleaned[df_cleaned.duplicated('id')]
print("Duplicate IDs:\n", duplicates)

Duplicate IDs:
 Empty DataFrame
Columns: [Unnamed: 0, id, difficulty, herotype, resource, rangetype, date, damage, toughness, control, mobility, utility]
Index: []


In [32]:
df_missing = df_cleaned[df_cleaned.isnull().any(axis=1)]
df_missing

,Unnamed: 0,id,difficulty,herotype,resource,rangetype,date,damage,toughness,control,mobility,utility
15,Bel'Veth,200.0,2,Fighter,NaN,Melee,2022-06-09,3,1,2,3,1
38,Garen,86.0,1,Fighter,NaN,Melee,2010-04-27,2,3,1,1,1
62,Katarina,55.0,2,Assassin,NaN,Melee,2009-09-19,3,1,1,3,1
112,Riven,92.0,2,Fighter,NaN,Melee,2011-09-14,3,2,2,3,1
152,Viego,234.0,3,Fighter,NaN,Melee,2021-01-21,3,1,2,2,1


In [33]:
df_cleaned['resource'] = df_cleaned['resource'].fillna('NoResource')
df_cleaned['resource'].isnull().sum()

0

In [34]:
print("🔍 Valori unice în 'herotype':")
print(df_cleaned['herotype'].unique())

print("\n🔍 Valori unice în 'resource':")
print(df_cleaned['resource'].unique())
print(df_cleaned['resource'].value_counts())

print("\n🔍 Valori unice în 'rangetype':")
print(df_cleaned['rangetype'].unique())


🔍 Valori unice în 'herotype':
['Fighter' 'Mage' 'Assassin' 'Marksman' 'Tank' 'Support']

🔍 Valori unice în 'resource':
['Blood Well' 'Mana' 'Energy' 'NoResource' 'Frenzy' 'Health' 'Rage'
 'Courage' 'Shield' 'Fury' 'Ferocity' 'Heat' 'Grit' 'Crimson Rush' 'Flow']
resource
Mana            142
Energy            6
NoResource        5
Rage              3
Fury              3
Health            2
Courage           2
Flow              2
Blood Well        1
Frenzy            1
Shield            1
Ferocity          1
Heat              1
Grit              1
Crimson Rush      1
Name: count, dtype: int64

🔍 Valori unice în 'rangetype':
['Melee' 'Ranged']


In [35]:
resource_counts = df_cleaned['resource'].value_counts()

# Pas 2: Selectăm valorile care apar o dată sau de două ori
rare_resources = resource_counts[resource_counts <= 3].index.tolist()

# Pas 3: Filtrăm campionii care au aceste valori
rare_champions = df_cleaned[df_cleaned['resource'].isin(rare_resources)]

# Afișăm
rare_champions[['Unnamed: 0', 'id', 'herotype', 'resource']]

,Unnamed: 0,id,herotype,resource
0,Aatrox,266.0,Fighter,Blood Well
19,Briar,233.0,Fighter,Frenzy
27,Dr. Mundo,36.0,Tank,Health
39,Gnar,150.0,Fighter,Rage
40,Mega Gnar,150.2,Fighter,Rage
68,Kled,240.0,Fighter,Courage
69,Kled & Skaarl,240.1,Fighter,Courage
86,Mordekaiser,82.0,Fighter,Shield
107,Rek'Sai,421.0,Fighter,Rage
110,Renekton,58.0,Fighter,Fury


In [36]:
df_cleaned = df_cleaned[~df_cleaned['Unnamed: 0'].isin(['Mega Gnar', 'Kled & Skaarl'])]
resources_to_keep = ['Mana', 'Energy', 'Fury']

df_cleaned['resource'] = df_cleaned['resource'].apply(
    lambda x: x if x in resources_to_keep else 'UniqueRes'
)

print("🔍 Valori unice în 'herotype':")
print(df_cleaned['herotype'].unique())

print("\n🔍 Valori unice în 'resource':")
print(df_cleaned['resource'].unique())
print(df_cleaned['resource'].value_counts())

print("\n🔍 Valori unice în 'rangetype':")
print(df_cleaned['rangetype'].unique())

🔍 Valori unice în 'herotype':
['Fighter' 'Mage' 'Assassin' 'Marksman' 'Tank' 'Support']

🔍 Valori unice în 'resource':
['UniqueRes' 'Mana' 'Energy' 'Fury']
resource
Mana         142
UniqueRes     19
Energy         6
Fury           3
Name: count, dtype: int64

🔍 Valori unice în 'rangetype':
['Melee' 'Ranged']


In [37]:
# Functie helper pentru a mapa orice coloană categorică cu valori numerice de la 1
def map_column_to_integers(df, column):
    unique_vals = sorted(df[column].unique())  # sortăm pentru consistență
    mapping = {val: idx + 1 for idx, val in enumerate(unique_vals)}
    df[column] = df[column].map(mapping)
    return mapping

# Aplicăm pe cele trei coloane
herotype_map = map_column_to_integers(df_cleaned, 'herotype')
resource_map = map_column_to_integers(df_cleaned, 'resource')
rangetype_map = map_column_to_integers(df_cleaned, 'rangetype')

# Afișăm mapările
print("✅ Mapping pentru 'herotype':", herotype_map)
print("✅ Mapping pentru 'resource':", resource_map)
print("✅ Mapping pentru 'rangetype':", rangetype_map)


✅ Mapping pentru 'herotype': {'Assassin': 1, 'Fighter': 2, 'Mage': 3, 'Marksman': 4, 'Support': 5, 'Tank': 6}
✅ Mapping pentru 'resource': {'Energy': 1, 'Fury': 2, 'Mana': 3, 'UniqueRes': 4}
✅ Mapping pentru 'rangetype': {'Melee': 1, 'Ranged': 2}


In [38]:
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])
df_cleaned = df_cleaned.sort_values(by='date')
df_test = df_cleaned.tail(2).copy()
df_cleaned = df_cleaned.iloc[:-2]

print("Training set size:", df_cleaned.shape)
print("Test set:")
df_test

Training set size: (168, 12)
Test set:


,Unnamed: 0,id,difficulty,herotype,resource,rangetype,date,damage,toughness,control,mobility,utility
5,Ambessa,799.0,3,2,1,1,2024-11-06,3,2,1,3,1
83,Mel,800.0,2,3,3,2,2025-01-23,3,1,2,1,1


## 🌲 Prediction using Random Forest (existing champions)


In [41]:

# 1. Preprocesare: sortăm după dată și separăm testul
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])
df_cleaned = df_cleaned.sort_values(by='date')

# Scoatem cei 2 campioni din final pentru test
df_test = df_cleaned.tail(2).copy()
df_cleaned = df_cleaned.iloc[:-2]

# Excludem complet variabila 'style' + celelalte coloane non-informative
excluded_cols = ['Unnamed: 0', 'id', 'date']  # style e complet scos
available_cols = [col for col in df_cleaned.columns if col not in excluded_cols]

# Variabilele target pe care vrem să le prezicem (fiecare pe rând)
target_cols = available_cols.copy()  # fiecare va fi target, restul vor fi input

# Pregătim output-ul
predictions = {}
true_values = {}

# 2. Loop prin fiecare coloană target
for col in target_cols:
    print(f"🔧 Antrenez model pentru: {col}")

    # Variabilele de input pentru acest model: tot ce nu e target-ul actual
    input_cols = [c for c in available_cols if c != col]

    # Input train/test
    X_train = df_cleaned[input_cols]
    X_test = df_test[input_cols]

    # Output train/test
    y_train = df_cleaned[col]
    y_test = df_test[col]

    # Model și antrenare
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Predicții
    preds = model.predict(X_test)

    # Salvăm rezultatele
    predictions[col] = preds
    true_values[col] = y_test.values

# 3. Afișăm rezultatele sub formă de dataframe
df_predictions = pd.DataFrame(predictions, index=df_test.index)
df_truth = pd.DataFrame(true_values, index=df_test.index)

print("🔮 Predicții pentru următorii 2 campioni:")
display(df_predictions)

print("🎯 Valori reale:")
display(df_truth)


🔧 Antrenez model pentru: difficulty
🔧 Antrenez model pentru: herotype
🔧 Antrenez model pentru: resource
🔧 Antrenez model pentru: rangetype
🔧 Antrenez model pentru: damage
🔧 Antrenez model pentru: toughness
🔧 Antrenez model pentru: control
🔧 Antrenez model pentru: mobility
🔧 Antrenez model pentru: utility
🔮 Predicții pentru următorii 2 campioni:


,difficulty,herotype,resource,rangetype,damage,toughness,control,mobility,utility
84,1,5,3,2,1,1,1,1,3
88,2,2,3,1,3,2,1,2,1


🎯 Valori reale:


,difficulty,herotype,resource,rangetype,damage,toughness,control,mobility,utility
84,1,5,3,2,1,1,2,3,3
88,1,1,3,1,3,1,1,3,1


## 🧠 Predicting future champions using clustering (KMeans + Random Forest)


In [42]:


# Reconstruim dataset-ul complet cu toți campionii cunoscuți (train + test)
df_all = pd.concat([df_cleaned, df_test], axis=0).reset_index(drop=True)

# Scoatem coloanele inutile + 'style'
excluded_cols = ['Unnamed: 0', 'id', 'date', 'style']
available_cols = [col for col in df_all.columns if col not in excluded_cols]

target_cols = available_cols.copy()

# Folosim toate datele ca train
X_full = df_all[available_cols]

# Creăm 5 "campioni viitori"
# Varianta: luăm media sau mod pentru fiecare coloană și adăugăm puțină variație (noise)
X_future = pd.DataFrame()

for col in available_cols:
    if df_all[col].dtype == 'object':
        # Pentru categorice — alegem valoarea cea mai frecventă
        most_common = df_all[col].mode()[0]
        X_future[col] = [most_common] * 5
    else:
        # Pentru numerice — adăugăm zgomot mic la medie
        mean_val = df_all[col].mean()
        noise = np.random.normal(0, 0.1, size=5)
        X_future[col] = mean_val + noise

# Asigurăm aceleași tipuri de date
for col in X_future.columns:
    X_future[col] = X_future[col].astype(df_all[col].dtype)

# Pregătim predicțiile
future_predictions = {}

for col in target_cols:
    input_cols = [c for c in available_cols if c != col]

    X_train = df_all[input_cols]
    y_train = df_all[col]

    X_pred = X_future[input_cols]

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_pred)

    future_predictions[col] = preds

# Convertim în DataFrame
df_future_predictions = pd.DataFrame(future_predictions)
print("🔮 Predicții pentru 5 campioni încă neintrați în joc:")
display(df_future_predictions)


🔮 Predicții pentru 5 campioni încă neintrați în joc:


,difficulty,herotype,resource,rangetype,damage,toughness,control,mobility,utility
0,1,2,3,2,3,2,2,1,2
1,1,2,3,2,3,2,2,1,2
2,1,2,3,2,3,2,1,2,1
3,1,2,3,2,3,2,1,2,1
4,1,2,3,2,3,2,2,1,2


In [43]:


# 1. Pregătim datele complete (inclusiv cei 2 campioni anteriori)
df_all = pd.concat([df_cleaned, df_test], axis=0).reset_index(drop=True)

# Excludem coloanele care nu ne ajută + 'style'
excluded_cols = ['Unnamed: 0', 'id', 'date', 'style']
available_cols = [col for col in df_all.columns if col not in excluded_cols]

# Coloanele pe care vrem să le prezicem
target_cols = available_cols.copy()

# 2. Aplicăm clustering pe campionii existenți
X_cluster = df_all[available_cols]

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X_cluster)

# 3. Luăm centrele de cluster ca „campioni viitori”
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=available_cols)

# Rotunjim valorile la cele mai apropiate întregi (dacă datele sunt discrete, cum pare)
cluster_centers = cluster_centers.round(0).astype(int)

# 4. Pregătim predicțiile
future_predictions = {}

for col in target_cols:
    input_cols = [c for c in available_cols if c != col]

    X_train = df_all[input_cols]
    y_train = df_all[col]

    X_pred = cluster_centers[input_cols]

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_pred)

    future_predictions[col] = preds

# 5. Rezultate
df_future_predictions = pd.DataFrame(future_predictions)
print("🔮 Predicții pentru 5 campioni viitori generați prin clustering:")
display(df_future_predictions)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


🔮 Predicții pentru 5 campioni viitori generați prin clustering:


,difficulty,herotype,resource,rangetype,damage,toughness,control,mobility,utility
0,3,3,3,2,3,1,2,1,2
1,2,5,3,2,2,1,2,1,3
2,1,6,3,1,1,3,3,2,1
3,2,2,3,1,3,1,1,3,1
4,1,2,3,1,3,3,2,1,1
